This code is for combine depth image paths and scale-based body weights information together and generate the csv file.  
Revised code is for ARC specifically.

In [ ]:
import os
os.getcwd()
import numpy as np
import pandas as pd


rootdir = "UrRootFolder"
DAY = "T1"
DAY_folder = rootdir + DAY
file_path = "UrScaleBWFolder"

if DAY == "T1":
    bw_csv = "ScaleBW.xlsx"
    bw_csv_path = file_path + bw_csv
    print(bw_csv_path)

bw_df = pd.read_excel(bw_csv_path) 
rowss = []
DAY_folder = DAY_folder + "/CSV_depth"
for pen in os.listdir(DAY_folder):
  if pen.startswith("pen"):
        print("Now is running pen number", pen)
        dep_folder = DAY_folder + "/" + pen + "/CSV_depth/"
        for bag_id in os.listdir(dep_folder):
            depthdir = dep_folder + bag_id + "/"
            images_per_bag = os.listdir(depthdir)
            rows = []
            try:
              weight_per_bag = bw_df["Weight (gr)"][bw_df["Top"] == bag_id + ".bag"].values[0]/1000
            except IndexError:
              continue
            
            for i, image_path in enumerate(images_per_bag):
                row = [depthdir+image_path, pen, bag_id, weight_per_bag]
                rows.append(row)
            rowss.append(rows)
  else:
        continue
  
col_vals = ['FilePath', 'Pen', "Bag_ID",'Weights']
labelled_depth = pd.concat(pd.DataFrame(rowss[i], columns=col_vals) for i in range(len(rowss)))
labelled_depth = labelled_depth.dropna(axis=0)
labelled_depth.to_csv("UrFolder/labelled_depth_" + DAY + ".csv", index=False)

## Add CV info into labelled_depth.csv files For CV2

In [18]:
import pandas as pd
import numpy as np
import random

def train_test_split(visit):
    # Load dataframes
    df1 = pd.read_csv("../labelled_depth_0718.csv")
    df2 = pd.read_csv("../labelled_depth_0801.csv")
    df3 = pd.read_csv("../labelled_depth_0815.csv")
    df4 = pd.read_csv("../labelled_depth_0829.csv")
    df5 = pd.read_csv("../labelled_depth_0912.csv")
    df6 = pd.read_csv("../labelled_depth_0927.csv")

    # Add 'Visit' column to each dataframe
    df1['Visit'] = '0718'
    df2['Visit'] = '0801'
    df3['Visit'] = '0815'
    df4['Visit'] = '0829'
    df5['Visit'] = '0912'
    df6['Visit'] = '0927'

    # Select dataframe based on visit value
    if visit == "0718":
        labelled_depth = df1
    elif visit == "0801":
        labelled_depth = df2
    elif visit == "0815":
        labelled_depth = df3
    elif visit == "0829":
        labelled_depth = df4
    elif visit == "0912":
        labelled_depth = df5
    elif visit == "0927":
        labelled_depth = df6
    else:
        raise ValueError(f"Invalid visit value: {visit}")

    # Assign visit value to the dataframe
    labelled_depth['Visit'] = str(visit)

    # Calculate number of unique pigs (assuming Bag_ID is your pig identifier)
    pig_n = np.unique(labelled_depth['Bag_ID']).shape[0]
    seed=42
    for rate in [0.75, 0.5, 0.25]:
        # Set random seed
        random.seed(seed)
        cv_name = "CrossValidation"+str(rate)
        # Initialize CrossValidation column with 'test'
        labelled_depth[cv_name] = 'test'
        train_bag_id = random.sample(list(np.unique(labelled_depth["Bag_ID"])), int(pig_n * rate))
        labelled_depth.loc[labelled_depth["Bag_ID"].isin(train_bag_id), cv_name] = 'train'

    # Save dataframe to CSV
    labelled_depth.to_csv(f'../labelled_depth_{visit}.csv', index=False)


# Call train_test_split for each visit
for visit in ['0829']:
    train_test_split(visit=visit)
